# Linear regression with scikit-learn 📈📈

Let's come back to yesterday's dataset and change the objective : we would like to predict the Salary as a function of the other variables.

To do so, we'll train a linear regression model and evaluate its performances. You will see that the preprocessings are quite similar to the ones from yesterday's template (except that the label encoding step is skipped here, because our label is already a numeric variable !). We simplified a bit the dataset so you can focus on modelisation and performance evaluation. We also added a few figures, because EDA is an important step before training any model : it allows to understand better the relationship between the different variables in the dataset 🤓

## What will you learn in this course? 🧐🧐

This lecture is a follow-along demo to walk you through building a linear regression model in practice using sklearn!

* EDA
* Baseline model : simple linear regression
    * Training pipeline
    * Test pipeline
* Multivariate linear regression
    * Training pipeline
    * Test pipeline


## EDA

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [4]:
# Import dataset
print("Loading dataset...")
dataset = pd.read_csv("/Users/qxzjy/vscworkspace/dsfs-ft-34/ml_module/courses/data/Data.csv")
print("...Done.")
print()

Loading dataset...
...Done.



In [5]:
# Basic stats
data_desc = dataset.describe(include='all')
print(dataset.shape)
data_desc

(10, 4)


,Country,Age,Salary,Purchased
count,10,9.000000,10.000000,10
unique,3,NaN,NaN,2
top,France,NaN,NaN,No
freq,4,NaN,NaN,5
mean,NaN,38.777778,64300.000000,NaN
std,NaN,7.693793,11681.419244,NaN
min,NaN,27.000000,48000.000000,NaN
25%,NaN,35.000000,55000.000000,NaN
50%,NaN,38.000000,64000.000000,NaN
75%,NaN,44.000000,71250.000000,NaN


- Taget variable : Y = 'Salary'
- Explanatory variables : 'Country', 'Age', 'Purchased'
- Number of examples : 10

- Imputation of missing values : 'Age'
- Normalisation : 'Age'
- OHE : 'Country', 'Purchased'

In [6]:
# Univariate analysis
# Distribution of each numeric variable
num_features = ['Age', 'Salary']
for i in range(len(num_features)):
    fig = px.histogram(dataset[num_features[i]])
    fig.show()

In [7]:
# Univariate analysis
# Barplot of each qualitative variable
cat_features = ['Country', 'Purchased']
for i in range(len(cat_features)):
    fig = px.bar(dataset[cat_features[i]])
    fig.show()

In [8]:
# Correlation matrix
corr_matrix = dataset[num_features].corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

There's a very strong correlation between the target and the Age variable ! This is a good news, it means that a linear model relating these two variables may be a good estimator of the salary 🙂

Let's analyze pairwise scatterplots to investigate the interdependencies between the variables :

In [9]:
# Visualize pairwise dependencies
fig = px.scatter_matrix(dataset)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=800, width = 800)
fig.show()

## Baseline model : simple linear regression
Let's try a first basic model : simple linear regression with only one feature. We choose the age because we just noticed that it is strongly correlated to the salary.

In [10]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = ["Age"]
target_variable = "Salary"

X = dataset[features_list]
y = dataset[target_variable]

print("...Done.")
print()

print('y : ')
print(y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

y : 
0    72000
1    48000
2    54000
3    61000
4    69000
Name: Salary, dtype: int64

X :
    Age
0  44.0
1  27.0
2  30.0
3  38.0
4  40.0


In [11]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



### Preprocessing

<Note type="note" title="Simplified syntax">

In this first example, we don't need to use the class `ColumnTransformer` because the preprocessing is very simple : it just consists in one step of imputation followed by a standardization, and these steps will be applied to the whole DataFrame (as there's actually only one column!). In this case, we can just create a `Pipeline` that describes the different steps and call the `fit_transform` and `transform` methods directly on it, by passing X_train/X_test as argument.

</Note>

In [ ]:
print("Preprocessing X_train...")
print(X_train.head())
print()
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
X_train = preprocessor.fit_transform(X_train)
print("...Done!")
print(X_train[0:5,:]) # X_train is now a numpy array
print() 

Preprocessing X_train...
    Age
4  40.0
9  37.0
1  27.0
6   NaN
7  48.0

...Done!
[[ 0.27978024]
 [-0.23673712]
 [-1.95846165]
 [-0.06456467]
 [ 1.65715986]]



In [13]:
# Test pipeline
print("Preprocessing X_test...")
print(X_test.head())
print()
X_test = preprocessor.transform(X_test)
print("...Done!")
print(X_test[0:5,:]) # X_test is now a numpy array
print() 

Preprocessing X_test...
    Age
2  30.0
8  50.0

...Done!
[[-1.44194429]
 [ 2.00150476]]



### Train model

In [14]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print("...Done.")

Train model...
...Done.


### Performance assessment

In [15]:
# Predictions on training set
print("Predictions on training set...")
y_train_pred = regressor.predict(X_train)
print("...Done.")
print(y_train_pred)
print()

Predictions on training set...
...Done.
[65666.0490968  61205.65076424 46337.65632237 62692.45020843
 77560.4446503  62692.45020843 71613.24687355 58232.05187587]



In [16]:
# Predictions on test set
print("Predictions on test set...")
y_test_pred = regressor.predict(X_test)
print("...Done.")
print(y_test_pred)
print()

Predictions on test set...
...Done.
[50798.05465493 80534.04353868]



In [17]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(y_train, y_train_pred))
print("R2 score on test set : ", r2_score(y_test, y_test_pred))

R2 score on training set :  0.7813729888409735
R2 score on test set :  0.9611572050845512


In [18]:
# Visualize the model
# Visualize predictions on training Set
fig = px.scatter(x = X_train.flatten().tolist(), y = y_train, title = "training set")
fig.add_trace(go.Scatter(x = X_train.flatten().tolist(), y = y_train_pred, name = "linear regression"))
fig.show()

# Visualize predictions on test Set
fig = px.scatter(x = X_test.flatten().tolist(), y = y_test, title = "test set")
fig.add_trace(go.Scatter(x = X_test.flatten().tolist(), y = y_test_pred, name = "linear regression"))
fig.show()

In the above example all assumptions needed to train a simple linear regression model are verified.

* There seems to be a linear relation between $Y$ and the feature $X$
* The deviation between the prediction and the actual values of $Y$, i.e. the residuals, seem to be comparable accross the values of $Y$ (this is Homoscedaticity).
* There does not seem to be any autocorrelation going on with the residuals, they seem to be randomly distributed accross all values of $Y$

In addition to that, our model seems to be a good predictor based on our training samples.

## Multivariate linear regression
Let's train a multivariate model by adding the categorical features : Country and Purchased.

In [19]:
# Separate target variable y from features X
print("Separating labels from features...")
features_list = ["Country", "Age", "Purchased"]
target_variable = "Salary"

X = dataset[features_list]
y = dataset[target_variable]

print("...Done.")
print()

print('y : ')
print(y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

y : 
0    72000
1    48000
2    54000
3    61000
4    69000
Name: Salary, dtype: int64

X :
   Country   Age Purchased
0   France  44.0        No
1    Spain  27.0       Yes
2  Germany  30.0        No
3    Spain  38.0        No
4  Germany  40.0       Yes


In [20]:
# Automatically detect names of numeric/categorical columns
numeric_features = ["Age"]
categorical_features = ["Country", "Purchased"]
# for i,t in X.dtypes.iteritems():
#     if ('float' in str(t)) or ('int' in str(t)) :
#         numeric_features.append(i)
#     else :
#         categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Age']
Found categorical features  ['Country', 'Purchased']


In [21]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



### Preprocessing

In [22]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # missing values will be replaced by columns' mean
    ('scaler', StandardScaler())
])

In [23]:
# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [24]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [25]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
   Country   Age Purchased
4  Germany  40.0       Yes
9   France  37.0       Yes
1    Spain  27.0       Yes
6    Spain   NaN        No
7   France  48.0       Yes
...Done.
[[ 0.27063731  1.          0.          1.        ]
 [-0.24603392  0.          0.          1.        ]
 [-1.96827133  0.          1.          1.        ]
 [ 0.          0.          1.          0.        ]
 [ 1.64842723  0.          0.          1.        ]]

Performing preprocessings on test set...
   Country   Age Purchased
2  Germany  30.0        No
8  Germany  50.0        No
...Done.
[[-1.4516001   1.          0.          0.        ]
 [ 1.99287472  1.          0.          0.        ]]



### Train model

In [26]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print("...Done.")

Train model...
...Done.


### Performance assessment

In [27]:
# Predictions on training set
print("Predictions on training set...")
y_train_pred = regressor.predict(X_train)
print("...Done.")
print(y_train_pred)
print()

Predictions on training set...
...Done.
[69000.         64690.01772977 46297.16942234 57628.38617903
 78907.85675999 57074.44439863 70297.16942234 62104.95608791]



In [28]:
# Predictions on test set
print("Predictions on test set...")
y_test_pred = regressor.predict(X_test)
print("...Done.")
print(y_test_pred)
print()

Predictions on test set...
...Done.
[52634.12773677 78484.74415536]



In [29]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(y_train, y_train_pred))
print("R2 score on test set : ", r2_score(y_test, y_test_pred))

R2 score on training set :  0.9016592604305448
R2 score on test set :  0.9470793284613951


### Conclusion about the model's quality
Here we can see that the R2-score has significantly improved when we went from simple to multiple linear regression. This is because we added some features that were indeed useful to predict the values of the target variables. 
Sometimes, adding more features won't necessarily improve the performances, and it can even deteriorate in some cases ! That's why in general, some feature selection methods are required to choose the right set of variables to be included into the model.

<Note type="note" title="What about plotting ?">
You can notice we didn't plot the multivariate model. It's simply because we can't ! To make a graphical representation of this model, we would need 4 dimensions (3 features + the target). Unfortunately, our brain is able to visualize only 3 at a time 🥴
This is the reason why it's so important to always compute some metrics such as the R2-score to assess the quality of our predictions: most of the times, we're "blind" on what's happening and we have to rely on these scores to make sure the model is making good predictions !
</Note>

### Interpreting the model's coefficients
As we've standardized our features, we can use the coefficients of the regression to estimate the importance of each feature for the prediction. The model's parameters are saved in a `.coef_` attribute:

In [30]:
regressor.coef_

array([ 7504.95148781,   432.38980745, -5467.54009813,  3440.56405393])

Each coefficient can be linked with the name of the corresponding feature by digging into the different pipelines that were used to produce the final version of the X_train/X_test arrays:

In [31]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Age', 'Country_Germany', 'Country_Spain', 'Purchased_Yes']


In [32]:
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Age,7504.951488
Country_Germany,432.389807
Country_Spain,-5467.540098
Purchased_Yes,3440.564054


<Note type="warning" title="Comparison in absolute values">
The feature importance is related to the absolute values of the coefficients. Don't forget to compute the absolute value before concluding !
</Note>

In [33]:
# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Country_Germany,432.389807
Purchased_Yes,3440.564054
Country_Spain,5467.540098
Age,7504.951488


In [34]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()